In [9]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [10]:
if not os.path.exists('log/'):
    os.makedirs('log/')

In [59]:
input_path = '../../input/'

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps, as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `parameters.xls` file. This file also includes scenarios.

In [49]:
kwargs = {'workers':2, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

#### Overview of all model params:
##### not assigned:
- demographic data
##### Exc 2:
- car_speed = bus_speed * 1.25
    - To-Do: Set car speed depending on road type!
- walk_speed = car_speed / 3
- sm.step_road_pathfinder(all_or_nothing=True, broken_modes=False, broken_routes=False)
- sm.step_pt_pathfinder(broken_modes=False, broken_routes=False, walk_on_road=False)
- Car usage has operating cost of 12ct/km 
-  Assume that a passenger makes two trips per day with a day ticket, sm.pt_los['price'] = 7.5 / 2
- We don't re-build the exact price structure, but make longer trips more expensive,  sm.pt_los.loc[sm.pt_los['length']>10000, 'price'] = 10 / 2
##### Exc 3:
- employment_rate = 0.7
- home_office_rate = 0.1
- work_trips_per_day = 1.05
- shopping_trips_per_day = 0.3
- car_ownership
- within zone travel: 120 seconds # with car thats too little! 
- preparation_logit(time=-0.00055, # beta_t
                     price=-1, # beta_p
                     transfers=0, # we don't want the number of transfers matter (even though it does sometimes)
                     mode=1, # theta_root
                     pt_mode=0.5, # theta_pt (nest parameter for PT)
                     pt_path=0.1, # theta_path (nest parameter for different paths of one PT option)
                     segments=sm.segments
    )
- matching of validation: validation_buffer, threshold

## Questions
- Discuss: What time period are we talking about? Only one hour or a complete day? Do we model return trips from work?
- should employment also be added to emission? (see page 10 quetzal 4 step presentation
- is parking time included? should it?
- dauert abbiegen länger als geradeaus fahren?
- where to best add mode of bike traffic?
- How to best set roads as unavailable for cars?
- How to best calibrate the model to make it realistic?
- add pendler verkehr from brandenburg
- bildet das modell stau ab?
- Focus on which part is most important?


To-Do:
- check for number of trips per inhabitant
- differentiate between POIs!
- make car ownership rate dependend on zone demographic data!
- make employment rate dependent on zone demographic data!
- add more categories to emission data (education etc)

- sm.centroids change with pop centroids

In [35]:
param_1 = [1,2,3]
param_2 = [50,60]
param_3 = [True, False]


In [61]:
param_combinations = []
n=0
for p_1 in param_1:
    for p_2 in param_2:
        for p_3 in param_3:
            param_combinations.append((n,(p_1, p_2, p_3)))
            n+=1


param_combinations
# save them to file
param_combinations_dict = dict((str(n),x) for n,x in param_combinations)
with open(input_path+'param_combinations_dict.txt', 'w') as f:
    print(param_combinations_dict, file=f)



In [79]:
# for each notebook read in param file based on scenario number
# configure parameters in notebooks based on file
import ast
with open(input_path+'param_combinations_dict.txt', "r") as file:
    param_1, param_2, param_3 = ast.literal_eval(file.read())["4"]
# file.close()


In [47]:
scenarios = [str(n) for n, x in param_combinations]

In [13]:
import time
start = time.time()

In [81]:
# parallel_call_notebook('001_test_environment.ipynb', arg_list=scenarios, **kwargs)


# Zone preparation


In [ ]:
# Zone creation
parallel_call_notebook('101_p_create_zones.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('102_OMS_POIs.ipynb', arg_list=scenarios, **kwargs)

# Network preparation


In [ ]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('Exercise1_network_creation.ipynb', arg_list=scenarios, **kwargs)


In [ ]:
# for scenario in scenarios:
#     parallel_call_notebook('Exercise2_OD-LoS-stack.ipynb', arg_list=[scenario], **kwargs)
#     parallel_call_notebook('Exercise3_Four_steps.ipynb', arg_list=[scenario], **kwargs)

In [ ]:
parallel_call_notebook('Exercise2_OD-LoS-stack.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('Exercise3_Four_steps.ipynb', arg_list=scenarios, **kwargs)

# Pathfinders and level of service

# Estimation

# Modelling

Except for the logit model step, all modelling steps require access to the OD volumes dataset.

In [ ]:
# Total execution time
end = time.time()
int(end - start)